## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [25]:
import pandas as pd

In [26]:
df=pd.read_csv('train.csv')

In [27]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
df.shape

(20800, 5)

In [29]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [30]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [31]:
###Drop Nan Values
df=df.dropna()


In [32]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [33]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [34]:
## Get the Dependent features
y=df['label']

In [35]:
X.shape

(18285, 4)

In [36]:
y.shape

(18285,)

In [37]:
import tensorflow as tf

In [38]:
tf.__version__

'2.12.0'

In [39]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [40]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [41]:
messages=X.copy()

In [42]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [43]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [44]:
#messages.reset_index(inplace=True)

In [45]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [46]:
import nltk
import re
from nltk.corpus import stopwords

In [47]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [49]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [50]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [51]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4080, 3426, 3241, 2902, 3644, 4056, 3060, 2839, 2150, 663],
 [1860, 2132, 477, 2617, 2976, 4856, 4703],
 [2694, 3147, 1021, 877],
 [2693, 1454, 3595, 3591, 1499, 1263],
 [4604, 2976, 731, 3797, 3078, 4045, 2976, 3344, 2814, 4535],
 [4539,
  2216,
  2257,
  4279,
  24,
  4369,
  4660,
  3025,
  3556,
  2131,
  243,
  3486,
  2320,
  2471,
  4703],
 [2020, 876, 2052, 2994, 3206, 339, 3015, 1430, 180, 483, 2579],
 [1510, 1569, 3486, 2584, 3425, 4171, 4369, 2687, 180, 483, 2579],
 [443, 4352, 876, 3482, 1665, 410, 4363, 125, 4369, 2158],
 [2353, 372, 3254, 3023, 2736, 2067, 2582, 4713],
 [1358, 654, 85, 4487, 3172, 466, 2293, 1133, 2794, 71, 644],
 [3591, 1151, 3644, 410, 4369, 3425],
 [342, 4558, 1715, 57, 62, 3048, 3531, 597, 4153],
 [1844, 1255, 1127, 425, 538, 1081, 1587, 180, 483, 2579],
 [2663, 3499, 1062, 4651, 3507, 180, 483, 2579],
 [2564, 797, 3867, 4310, 3842, 3156, 2409, 240, 4672, 1779],
 [794, 334, 2132],
 [2848, 919, 705, 4386, 4369, 862, 3225, 4703],
 [4075, 637, 477, 204

In [52]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [53]:
onehot_repr[1]

[1860, 2132, 477, 2617, 2976, 4856, 4703]

### Embedding Representation

In [54]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4080 3426 3241 ...    0    0    0]
 [1860 2132  477 ...    0    0    0]
 [2694 3147 1021 ...    0    0    0]
 ...
 [1216 1684 4252 ...    0    0    0]
 [4388 3425 3402 ...    0    0    0]
 [3084 1766 2268 ...    0    0    0]]


In [55]:
embedded_docs[1]

array([1860, 2132,  477, 2617, 2976, 4856, 4703,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [56]:
embedded_docs[0]

array([4080, 3426, 3241, 2902, 3644, 4056, 3060, 2839, 2150,  663,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [57]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [58]:
len(embedded_docs),y.shape

(18285, (18285,))

In [59]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [60]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [69]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 15s 64ms/step - loss: 0.3243 - accuracy: 0.8383 - val_loss: 0.2004 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 3s 16ms/step - loss: 0.1497 - accuracy: 0.9430 - val_loss: 0.1952 - val_accuracy: 0.9196
Epoch 3/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1091 - accuracy: 0.9594 - val_loss: 0.2247 - val_accuracy: 0.9142
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0839 - accuracy: 0.9713 - val_loss: 0.2739 - val_accuracy: 0.9115
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0656 - accuracy: 0.9772 - val_loss: 0.2528 - val_accuracy: 0.9031
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0528 - accuracy: 0.9835 - val_loss: 0.2980 - val_accuracy: 0.9127
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0455 - accuracy: 0.9864 - val_loss: 0.3594 - val_accuracy: 0.9109
E

### Adding Dropout 

In [68]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [70]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 2ms/step


In [71]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [72]:
from sklearn.metrics import confusion_matrix

In [73]:
confusion_matrix(y_test,y_pred)

array([[3132,  287],
       [ 275, 2341]])

In [74]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9068765534382767

In [75]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.89      0.89      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

